In [1]:
pip install transformers

In [10]:
import torch
print(torch.cuda.is_available())  # Should be True
print(torch.cuda.get_device_name(0))  # Shows GPU name

True
Tesla T4


In [3]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

In [4]:
dataset = load_dataset("imdb")
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

plain_text/test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

plain_text/unsupervised-00000-of-00001.p(…):   0%|          | 0.00/42.0M [00:00<?, ?B/s]

...

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [5]:
def tokenize(batch):
  return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=512)

In [6]:
tokenized_dataset = dataset.map(tokenize, batched=True)
tokenized_dataset.set_format("torch",columns=["input_ids", "attention_mask", "label"])

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [7]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english",num_labels = 2)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [8]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    save_strategy="epoch",
    logging_dir = "./logs",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    learning_rate=2e-5,  # or use `accelerate` to auto-detect
    fp16=True
    )

In [9]:
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_dataset["train"],
    eval_dataset = tokenized_dataset["test"],
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
)

In [11]:
trainer.train()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: kavanapds00 (kavanapds00-self) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
500,0.265300
1000,0.232900
1500,0.214200
2000,0.158000
2500,0.139500
3000,0.139100
3500,0.088400
4000,0.078000
4500,0.078900
5000,0.056500


TrainOutput(global_step=7815, training_loss=0.10376144636157836, metrics={'train_runtime': 1556.3151, 'train_samples_per_second': 80.318, 'train_steps_per_second': 5.021, 'total_flos': 1.6558424832e+16, 'train_loss': 0.10376144636157836, 'epoch': 5.0})

In [15]:
trainer.save_model("distilbert-imdb-finetuned")


In [19]:
model.push_to_hub("KavanaPadaki/distilbert-imdb")

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...nbi3mtv/model.safetensors:   0%|          |  575kB /  268MB            

CommitInfo(commit_url='https://huggingface.co/KavanaPadaki/distilbert-imdb/commit/c2ff0781e83b69261e47cd8a843dabcc5b790ea5', commit_message='Upload DistilBertForSequenceClassification', commit_description='', oid='c2ff0781e83b69261e47cd8a843dabcc5b790ea5', pr_url=None, repo_url=RepoUrl('https://huggingface.co/KavanaPadaki/distilbert-imdb', endpoint='https://huggingface.co', repo_type='model', repo_id='KavanaPadaki/distilbert-imdb'), pr_revision=None, pr_num=None)

In [ ]:
pipeline("text-classification", model="path-or-hub-name")

In [22]:
import evaluate
import numpy as np

accuracy_metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return accuracy_metric.compute(predictions=predictions, references=labels)

In [23]:
predictions = trainer.predict(tokenized_dataset["test"])
acc = accuracy_metric.compute(
    predictions=np.argmax(predictions.predictions, axis=-1),
    references=predictions.label_ids
)
print(f"Validation Accuracy: {acc['accuracy']:.4f}")

Validation Accuracy: 0.9344


In [24]:
f1_metric = evaluate.load("f1")
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")

In [26]:
f1_score = f1_metric.compute(predictions=np.argmax(predictions.predictions, axis=-1), references=predictions.label_ids)
precision_score = precision_metric.compute(predictions=np.argmax(predictions.predictions, axis=-1), references=predictions.label_ids)
recall_score = recall_metric.compute(predictions=np.argmax(predictions.predictions, axis=-1), references=predictions.label_ids)

In [27]:
print(f"Validation F1 Score: {f1_score['f1']:.4f}")
print(f"Validation Precision Score: {precision_score['precision']:.4f}")
print(f"Validation Recall Score: {recall_score['recall']:.4f}")

Validation F1 Score: 0.9350
Validation Precision Score: 0.9267
Validation Recall Score: 0.9434


In [29]:
%%writefile app.py
import streamlit as st
from transformers import pipeline

# Load your fine-tuned model from local folder or Hugging Face Hub
MODEL_NAME = "KavanaPadaki/distilbert-imdb"  # or "./distilbert-imdb-finetuned"
classifier = pipeline("text-classification", model=MODEL_NAME)

# Streamlit UI
st.title("🎬 IMDb Sentiment Classifier")
st.write("Enter a movie review and see if it's positive or negative.")

# Text input
user_input = st.text_area("Movie Review", height=150)

if st.button("Classify"):
    if user_input.strip():
        result = classifier(user_input, truncation=True, max_length=512)[0]
        label = result['label']
        score = result['score']
        st.markdown(f"**Prediction:** {label}")
        st.markdown(f"**Confidence:** {score:.2%}")
    else:
        st.warning("Please enter a review before classifying.")

Writing app.py
